importing libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Data1.xlsx to Data1.xlsx


reading file, creating a new dataframe

In [3]:
df = pd.read_excel('Data1.xlsx')

In [4]:
#constant declarations

threshold_const = 10 #relaxation period in secs i.e. (10 secs of error is allowed b/w avg rt and individual rt) (rt = response time)
num_of_students = 46
num_of_ques = 15 #number of questions for each student, each question is worth 1 mark

In [5]:
df.info() # to see the number of entries and if all the variables have equal no. of data points

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Username/ID             690 non-null    object
 1   Question                690 non-null    object
 2   Question Difficulty     690 non-null    object
 3   Chosen Option           690 non-null    object
 4   Answer Submission Time  690 non-null    object
 5   Answer Attempts         690 non-null    int64 
 6   Time Spent (secs)       690 non-null    int64 
 7   Score Earned            690 non-null    int64 
 8   State                   690 non-null    object
dtypes: int64(3), object(6)
memory usage: 48.6+ KB


creating a seperate dataframe

the variable names should be the same in the original dataframe (like 'Username/ID' should be the variable name for the name of the students in every dataframe that is read)


In [6]:
#creating a new df
data = [df['Username/ID'], df['Question'], df['Question Difficulty'], df['Time Spent (secs)'], df['Score Earned']]
headers = ['Username/ID','Question', 'Question Difficulty', 'time_spent(secs)', 'score_earned']

new_df = pd.concat(data, axis=1, keys=headers)

In [ ]:
#turning string to int
new_df['Question'][new_df['Question'] == 'Q1'] = 1
new_df['Question'][new_df['Question'] == 'Q2'] = 2
new_df['Question'][new_df['Question'] == 'Q3'] = 3
new_df['Question'][new_df['Question'] == 'Q4'] = 4
new_df['Question'][new_df['Question'] == 'Q5'] = 5
new_df['Question'][new_df['Question'] == 'Q6'] = 6
new_df['Question'][new_df['Question'] == 'Q7'] = 7
new_df['Question'][new_df['Question'] == 'Q8'] = 8
new_df['Question'][new_df['Question'] == 'Q9'] = 9
new_df['Question'][new_df['Question'] == 'Q10'] = 10
new_df['Question'][new_df['Question'] == 'Q11'] = 11
new_df['Question'][new_df['Question'] == 'Q12'] = 12
new_df['Question'][new_df['Question'] == 'Q13'] = 13
new_df['Question'][new_df['Question'] == 'Q14'] = 14
new_df['Question'][new_df['Question'] == 'Q15'] = 15
### clear output

In [10]:
name_list = new_df['Username/ID'].unique().tolist() #creating a list of student names for later use

In [11]:
name_list.sort() #alphabetically ordered list of student names

#### Now we find,

1. time spent(for each difficulty level)
2. time spent(for each difficulty level) and (correct response)
3. score earned(for each difficulty level)
4. score earned(for each difficulty level) and (correct response)



In [8]:
new_df.loc[((new_df['Username/ID'] == 'MUHAMMED FAIES') & (new_df['Question Difficulty'] == 'low'))]['time_spent(secs)'].sum()

109

the code above gives the time_spent(secs) total by MUHAMMED FAIES for low difficulty questions. Using for loop we'll find time_spent(secs) for each difficulty for each student. Then the same with an added condition that the responses are correct.

In [12]:
low_diff_rt = []
mid_diff_rt = []
high_diff_rt = []

low_diff_score = []
mid_diff_score = []
high_diff_score = []

low_diff_rt_correct = []
mid_diff_rt_correct = []
high_diff_rt_correct = []

for name in name_list:
  # total response time for the each difficulty for each student
  low_diff_rt.append(new_df.loc[((new_df['Username/ID'] == name) & (new_df['Question Difficulty'] == 'low'))]['time_spent(secs)'].sum())
  mid_diff_rt.append(new_df.loc[((new_df['Username/ID'] == name) & (new_df['Question Difficulty'] == 'medium'))]['time_spent(secs)'].sum())
  high_diff_rt.append(new_df.loc[((new_df['Username/ID'] == name) & (new_df['Question Difficulty'] == 'high'))]['time_spent(secs)'].sum())

  # total scores for the each difficulty for each student
  low_diff_score.append(new_df.loc[((new_df['Username/ID'] == name) & (new_df['Question Difficulty'] == 'low'))]['score_earned'].sum())
  mid_diff_score.append(new_df.loc[((new_df['Username/ID'] == name) & (new_df['Question Difficulty'] == 'medium'))]['score_earned'].sum())
  high_diff_score.append(new_df.loc[((new_df['Username/ID'] == name) & (new_df['Question Difficulty'] == 'high'))]['score_earned'].sum())

  # total response times for each difficulty for each student given the answer is correct
  low_diff_rt_correct.append(new_df.loc[((new_df['Username/ID'] == name) & (new_df['Question Difficulty'] == 'low') & (new_df['score_earned'] == 1))]['time_spent(secs)'].sum())
  mid_diff_rt_correct.append(new_df.loc[((new_df['Username/ID'] == name) & (new_df['Question Difficulty'] == 'medium') & (new_df['score_earned'] == 1))]['time_spent(secs)'].sum())
  high_diff_rt_correct.append(new_df.loc[((new_df['Username/ID'] == name) & (new_df['Question Difficulty'] == 'high') & (new_df['score_earned'] == 1))]['time_spent(secs)'].sum())

#### finding number of flagged response times (for each student)

first, we find the avg resp time of each question

In [13]:
class_avg_resp_time = []#list will contain the average response time for each question, since there are 15 ques then this list will have 15 values

for num in range(1,(num_of_ques+1)):
  class_avg_resp_time.append(new_df.loc[new_df['Question'] == num]['time_spent(secs)'].mean())

In [14]:
class_avg_resp = class_avg_resp_time * num_of_students #making the list the same length as the dataframe (e.g-> [1,2,3] * 2 gives [1,2,3,1,2,3])

In [15]:
new_df.insert(loc=5, column='class average response time', value=class_avg_resp)

second step, we create a function to flag out individual response times greater than the average response time by at least the margin of the threshold count (i.e. 10 in this case)

In [16]:
def func(x, y):
  if x > y + threshold_const:
    return 1
  else:
    return 0

In [17]:
new_df['flag'] = new_df.apply(lambda x: func(x['time_spent(secs)'], x['class average response time']), axis=1)

creating a seperate dataframe (df1) using two conditions and groupby method to single out the list of 'no.' of flags where the response is correct

In [18]:
df1 = new_df.loc[(new_df['score_earned'] == 1) & (new_df['flag'] == 1)]

In [19]:
df1 = df1.groupby(['Username/ID']).sum()

<ipython-input-19-c7192b42c8aa>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df1 = df1.groupby(['Username/ID']).sum()


In [20]:
df1 = df1.reset_index()

In [21]:
flag_correct_lst = df1['flag'].tolist()
# this list only has the students which satisfy both the conditions (a) score = 1 (b) flag = 1, but there are students that dont have a flag AND a correct response together.
# so, the following code adds a 0 in the resp. places

In [22]:
#name_list ###list of entire students
#df1['Username/ID'].tolist() ###list of students that satisfy both conditions somewhere in any question

In [23]:
#name_list.index('SHIVIKA NA') #one of the students who dont have any flag with a correct response. 35 is the position of the student in the entire name_list.

In [24]:
no_correct_flags = list(set(name_list) - set(df1['Username/ID'].tolist())) #this list is the difference, it contains the names of the students like shivika that havent satisfied the two conditions for any ques.
posn = []
for x in no_correct_flags:#iterating through the list of students like shivika and finding the index/position
  posn.append(name_list.index(x))

In [25]:
for p in posn:
  flag_correct_lst.insert(p, 0)

In [26]:
## what we're doing here is basically inserting a zero, where the student doesn't have a flag with a correct response
## eg. shivika na should have a zero under the flag_correct column because none of her correct attempts took 10 secs more than the average response time

In [27]:
len(flag_correct_lst)

46

In [28]:
#df1 has two conditions applied to it: (a) score earned = 1 and (b) flag = 1
#we'll take out the flag list from this dataframe, which is the correct response flag list

creating a seperate dataframe (df2) to make a list of all the other variables (entire time spent(each student), score earned, all flagged)

In [29]:
df2 = new_df.groupby(['Username/ID']).sum()

<ipython-input-29-5f17228a0e6e>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df2 = new_df.groupby(['Username/ID']).sum()


In [30]:
df2 = df2.reset_index()

In [31]:
# we have had to create two seperate dataframes to make the lists out of, because
# there's a condition we need to satisfy (score_earned = 1) for the flagged_rt_correct list.
# we don't want that condition to be applied to other variables.

In [32]:
time_spent_lst = df2['time_spent(secs)'].tolist()
score_earned_lst = df2['score_earned'].tolist()
flag_lst = df2['flag'].tolist()

creating the final dataframe

The lists that are being added to the final dataframe
1. name_list (alphabetically)
2. time_spent(secs) (each students entire time spent)
3. score_earned(total score of each student, out of 15)

> total response times wrt difficulty of each student (one value, e.g -> muhammad = 109 secs)
4. low_diff_rt
5. mid_diff_rt
6. high_diff_rt

> scores wrt difficulty of each student
7. low_diff_score
8. mid_diff_score
9. high_diff_score

> total response times wrt difficulty of each student given the answers are correct
10. low_diff_rt_correct
11. mid_diff_rt_correct
12. high_diff_rt_correct


16. flagged response times i.e. number of times that student has been flagged
17. flagged rt when correct

In [33]:
name_list.sort()

In [35]:
final_df = pd.DataFrame(list(zip(name_list, time_spent_lst, score_earned_lst, flag_lst, flag_correct_lst, low_diff_rt, mid_diff_rt, high_diff_rt, low_diff_rt_correct,
                                 mid_diff_rt_correct, high_diff_rt_correct, low_diff_score, mid_diff_score, high_diff_score)),
               columns =['Student', 'total time spent(secs)', 'total score earned', 'flag', 'flag when correct', 'low_diff_rt', 'mid_diff_rt', 'high_diff_rt',
                         'low_diff_rt_correct', 'mid_diff_rt_correct', 'high_diff_rt_correct', 'low_diff_score', 'mid_diff_score', 'high_diff_score'])

In [38]:
final_df.shape # no. of students and the resp variables (13 + 1(names))

(46, 14)

In [39]:
final_df.head(10)

,Student,total time spent(secs),total score earned,flag,flag when correct,low_diff_rt,mid_diff_rt,high_diff_rt,low_diff_rt_correct,mid_diff_rt_correct,high_diff_rt_correct,low_diff_score,mid_diff_score,high_diff_score
0,ADITI SUKHIJA,948,15,6,6,228,332,388,228,332,388,7,5,3
1,ANJALI ARORA,924,12,7,6,167,367,390,167,192,390,7,2,3
2,ANJALI NA,1492,11,9,6,612,647,233,612,224,132,7,3,1
3,Aditya P,1034,11,8,6,247,431,356,188,346,285,5,4,2
4,BHAWNA NA,306,12,1,1,131,103,72,126,103,72,4,5,3
5,DEEPESH KUMAR,482,10,2,2,124,231,127,91,202,58,6,3,1
6,DISHA GUPTA,929,15,6,6,222,323,384,222,323,384,7,5,3
7,DIVYA GOEL,735,12,4,3,179,251,305,179,251,161,7,4,1
8,GUR RAKHA,1492,14,7,6,365,289,838,365,289,639,7,5,2
9,HARSHITA MAITREY,1157,7,5,0,381,382,394,67,76,0,5,2,0


now to round it all up, we have in the final dataset:

1.students

2.total time spent on the paper (15 questions)

3.total score (out of 15)

4.flags (if their response times are +-10 of the average response time)*

5.flagged when correct (same as flags but question answered correctly)

6.low/mid/high_diff_rt: collective response time of each student for each difficulty level

7.low/mid/high_diff_rt_correct: same as above but the questions are answered correctly

we'll be trying to use this data for clustering